In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [1]:
import pickle
with open('features.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [2]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [3]:
# !wget https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from google.colab import drive
from helper_prabowo_ml import clean_html, remove_links, non_ascii, lower, email_address, removeStopWords, punct, remove_
import re

2023-08-23 10:16:29.558441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 10:16:36.950862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec22046/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def preproc(df, colname):
  df[colname] = df[colname].apply(func=clean_html)
  df[colname] = df[colname].apply(func=remove_links)
  df[colname] = df[colname].apply(func=non_ascii)
  df[colname] = df[colname].apply(func=lower)
  df[colname] = df[colname].apply(func=email_address)
  # df[colname] = df[colname].apply(func=removeStopWords)
  df[colname] = df[colname].apply(func=punct)
  df[colname] = df[colname].apply(func=remove_)
  return(df)

In [6]:
data2.keys()

dict_keys(['speech', 'text', 'label'])

In [7]:
X = data2['text']
Y = data2['label']

In [8]:
df = pd.DataFrame(list(zip(X, Y)), columns = ['Text','Labels'])

In [9]:
df_clean = preproc(df, 'Text')
# df_clean.drop('index', axis=1, inplace=True)
df_clean['num_words'] = df_clean['Text'].apply(lambda x: len(x.split()))

In [10]:
df_clean['Labels'] = pd.Categorical(df_clean['Labels'])
df_clean['Labels'] = df_clean['Labels'].cat.codes

In [11]:
# ! pip install transformers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from transformers import AutoTokenizer, TFBertModel

# df_train, df_test = train_test_split(df_clean, test_size=0.3, random_state=42,
#                                      stratify=df_clean['Labels'])

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

max_len = 100

/home/ec22046/anaconda3/envs/msc_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

: 

In [ ]:
X = tokenizer(
    text=df['Text'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)


In [ ]:
import tensorflow as tf

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X_train['input_ids'],X_train['attention_mask'],df_clean['Labels']))

In [ ]:
# X_train['input_ids'].shape,X_train['attention_mask'].shape

In [ ]:
train_size = int(0.8 * len(df['Text']))
test_size = len(df['Text'])-train_size
print(train_size,test_size)

In [ ]:
X_train_input_ids = X['input_ids'][0:train_size,:]
X_train_att = X['attention_mask'][0:train_size,:]
Y_train = df_clean['Labels'].loc[:train_size-1]

In [ ]:
X_test_input_ids = X['input_ids'][train_size:,:]
X_test_att = X['attention_mask'][train_size:,:]
Y_test = df_clean['Labels'].loc[train_size:]

In [ ]:
# X_train_input_ids.shape,X_train_att.shape,Y_train.shape

In [ ]:
# X_test_input_ids.shape,X_test_att.shape,Y_test.shape

In [ ]:
# X_test['input_ids'].shape,X_test['attention_mask'].shape

In [ ]:

import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

In [ ]:
embeddings = bert(input_ids, attention_mask = input_mask)[0] # 0 = last hidden state, 1 = poller_output
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(128, activation='relu')(out)

y = Dense(4, activation='softmax')(out)

In [ ]:
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
)


In [ ]:
loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)
model.summary()

In [ ]:
history = model.fit(
    x = {'input_ids':X_train_input_ids, 'attention_mask':X_train_att},
    y = to_categorical(Y_train),
    validation_data = ({'input_ids':X_test_input_ids, 'attention_mask':X_test_att},
                        to_categorical(Y_test)),
    epochs=25,
    batch_size=32
)